In [1]:
import pandas as pd
import numpy as np
import sys, os , time
from tabulate import tabulate

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(folder+file_name)
    print(tabulate(table, headers='keys', tablefmt='rst')) #latex

In [2]:
!ls

CIFAR	       final_CIFAR  LabelMe_v3	Plot_Results.ipynb  run_LabelMe.sh
CIFAR_tol3     LabelMe	    new_CIFAR	run_CIFAR.sh	    run_simple.sh
CIFAR_tol3_v2  LabelMe_v2   old_exec	run_CIFAR_v2.sh     simple


In [3]:
!ls final_CIFAR

result_scenario1.txt		    simCIFAR_OursGlobal_trainAnn_s1.csv
result_scenario4.txt		    simCIFAR_OursGlobal_trainAnn_s4.csv
result_scenario5.txt		    simCIFAR_OursGlobal_trainAnn_s7.csv
result_scenario7.txt		    simCIFAR_OursGlobal_train_s1.csv
simCIFAR_DS_test_s1.csv		    simCIFAR_OursGlobal_train_s4.csv
simCIFAR_DS_test_s4.csv		    simCIFAR_OursGlobal_train_s5.csv
simCIFAR_DS_test_s7.csv		    simCIFAR_OursGlobal_train_s7.csv
simCIFAR_DS_train_s1.csv	    simCIFAR_Raykar_test_s1.csv
simCIFAR_DS_train_s4.csv	    simCIFAR_Raykar_test_s4.csv
simCIFAR_DS_train_s7.csv	    simCIFAR_Raykar_test_s7.csv
simCIFAR_hardMV_test_s1.csv	    simCIFAR_Raykar_trainAnn_s1.csv
simCIFAR_hardMV_test_s4.csv	    simCIFAR_Raykar_trainAnn_s4.csv
simCIFAR_hardMV_test_s5.csv	    simCIFAR_Raykar_trainAnn_s7.csv
simCIFAR_hardMV_test_s7.csv	    simCIFAR_Raykar_train_s1.csv
simCIFAR_hardMV_train_s1.csv	    simCIFAR_Raykar_train_s4.csv
simCIFAR_hardMV_train_s4.csv	    simCIFAR_Raykar_train_s7.csv
simCIFAR_hardMV_train

In [4]:
folder = "final_CIFAR/"
files = sorted(os.listdir(folder))

In [10]:
for s in range(1,10):
    print("\n****************************** Scenario "+str(s)+" ******************************")
    scenario_files = find_match("s"+str(s)+".csv",files)
    
    #all from softMV
    aux = find_match("softMV",scenario_files)
    for value in aux:
        plot_files(value)

    #all from hardMV
    aux = find_match("hardMV",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("DS",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("Raykar",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("OursGlobal",scenario_files)
    for value in aux:
        plot_files(value)        
    
    aux = find_match("Ours2",scenario_files)
    for value in aux:
        plot_files(value)
    
    aux = find_match("Ours3",scenario_files)
    for value in aux:
        plot_files(value)


****************************** Scenario 1 ******************************

File to show:  simCIFAR_softMV_test_s1.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global           0.50558       0.50558
====  ============  ==========  ============

File to show:  simCIFAR_softMV_train_s1.csv
====  ============  ==========  ============  ============  =========  =============  ==========
  ..  Unnamed: 0      Accuracy    F1 (micro)    Mean NormF    Mean JS    Wmean NormF    Wmean JS
====  ============  ==========  ============  ============  =========  =============  ==========
   0  Global          0.513692      0.513692      0.147445   0.234836       0.147411     0.23472
====  ============  ==========  ============  ============  =========  =============  ==========

File to show:  simCIFAR_hardMV_test_s1.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (

In [16]:
methods_list = ["Ann(tr)","softMV","hardMV", "DS", "Raykar", "OursGlobal", ""]
methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","",]

def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(folder+archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return np.mean(result_list)
    except:
        return "-"

def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    for s in range(1,10):
        t["Scenario "+str(s)] = ["-" for _ in range(len(methods_list))]
        if tipo == 'acc':
            t.loc[0,"Scenario "+str(s)] =  calculate_aux("result_scenario"+str(s)+".txt",what_to_do="mv")

        scenario_files = find_match("s"+str(s)+".csv",files)

        for k, method in enumerate(methods_list[1:]):
            aux = find_match(method,scenario_files)
            if tipo == "acc": #como normal?
                test_aux = find_match("_test_",aux)
                if len(test_aux) != 0:
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+test_aux[0])["Accuracy"][0], 4)
                    
            elif tipo == 'norm' or tipo=='js':
                kl_aux = find_match("_train_",aux)
                if len(kl_aux) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    if tipo == 'norm':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"NormF"][0] ,4)
                        except:
                            pass
                    elif tipo=='js':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"JS"][0] ,4)
                        except:
                            pass
            elif tipo == 'accann':
                train_ann = find_match("_trainAnn_",aux)
                if len(train_ann) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_ann[0])[to_extract+"ACC imiting Annot"][0] ,4)
            elif tipo == 'rmse':
                train_aux = find_match("_train_",aux)
                if len(train_aux) != 0:
                    try:
                        t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_aux[0])["Average RMSE"][0] ,4)
                    except:
                        pass
            elif tipo == "epochs":
                t.loc[k+1,"Scenario "+str(s)]=calculate_aux("result_scenario"+str(s)+".txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_norm_mean = compare_table("norm","mean")
t_js_mean = compare_table("js","mean")
t_norm_wmean = compare_table("norm","wmean")
t_js_wmean = compare_table("js","wmean")
t_acc

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),0.73942,-,-,0.73802,0.85384,-,0.73418,-,-
1,softMV,0.5056,-,-,0.5089,0.555,-,0.4869,-,-
2,hardMV,0.6747,-,-,0.7041,0.7464,-,0.6756,-,-
3,DS,0.7454,-,-,0.7037,-,-,0.6813,-,-
4,Raykar,0.7797,-,-,0.7659,-,-,0.763,-,-
5,OursGlobal,0.7311,-,-,0.7393,0.7389,-,0.7638,-,-
6,,0.7454,-,-,0.7037,0.7389,-,0.6813,-,-


In [12]:
t_js_mean #normal

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.2348,-,-,0.2307,0.212,-,0.2144,-,-
2,hardMV,0.1938,-,-,0.1984,0.2369,-,0.1448,-,-
3,DS,0.0044,-,-,0.0948,-,-,0.1087,-,-
4,Raykar,0.0042,-,-,0.0569,-,-,0.0475,-,-
5,OursGlobal,0.1966,-,-,0.2339,0.208,-,0.125,-,-
6,,0.0044,-,-,0.0948,0.208,-,0.1087,-,-


In [17]:
t_norm_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1474,-,-,0.145,0.1375,-,0.1398,-,-
2,hardMV,0.1376,-,-,0.1406,0.1566,-,0.1225,-,-
3,DS,0.014,-,-,0.0638,-,-,0.0674,-,-
4,Raykar,0.013,-,-,0.0583,-,-,0.0559,-,-
5,OursGlobal,0.1401,-,-,0.1548,0.1383,-,0.1109,-,-
6,,0.014,-,-,0.0638,0.1383,-,0.0674,-,-


In [20]:
t_accAnn_mean = compare_table("accann","mean")
t_accAnn_wmean = compare_table("accann","wmean")
t_rmseAnn = compare_table("rmse","normal")
t_accAnn_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,0.6856,-,-,0.7567,0.8267,-,0.7005,-,-
5,OursGlobal,0.6392,-,-,0.6807,0.741,-,0.6494,-,-
6,,0.6392,-,-,0.6807,0.741,-,0.6494,-,-


In [35]:
t_epochs = compare_table("epochs")
t_epochs

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [25]:
print(tabulate(t_acc,headers='keys',tablefmt='latex'))

\begin{tabular}{rlrrrrrrrll}
\hline
    & Methods    &   Scenario 1 &   Scenario 2 &   Scenario 3 &   Scenario 4 &   Scenario 5 &   Scenario 6 &   Scenario 7 & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    &     0.777371 &      0.61335 &      0.78022 &     0.771673 &     0.807082 &     0.622711 &     0.782255 & -            & -            \\
  1 & softMV     &     0.6644   &      0.6081  &      0.6572  &     0.6667   &     0.6764   &     0.5914   &     0.665    & -            & -            \\
  2 & hardMV     &     0.7248   &      0.5947  &      0.7527  &     0.7345   &     0.7297   &     0.5784   &     0.7913   & -            & -            \\
  3 & DS         &     0.9482   &      0.9429  &      0.9464  &     0.9344   &     0.9417   &     0.8413   &     0.9254   & -            & -            \\
  4 & Raykar     &     0.9415   &      0.9418  &      0.9417  &     0.9372   &     0.9308   &     0.8037   &     0.9179   & -            & -            \\
  5 & OursGlobal &     0.90

In [27]:
t_js_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1748,0.1881,0.1131,0.1812,0.1801,0.1937,0.1296,-,-
2,hardMV,0.1835,0.1966,0.1502,0.1537,0.1773,0.1981,0.1482,-,-
3,DS,0.0029,0.0036,0.0046,0.1155,0.531,0.1394,0.1375,-,-
4,Raykar,0.0066,0.0075,0.0055,0.0905,0.1385,0.1055,0.0738,-,-
5,OursGlobal,0.1511,0.1574,0.1466,0.1346,0.1181,0.1377,0.1271,-,-
6,Ours2,-,-,-,-,-,-,-,-,-
7,Ours3,-,-,-,-,-,-,-,-,-


In [31]:
print(tabulate(t_js_mean,headers='keys',tablefmt='latex'))

\begin{tabular}{rllllllllll}
\hline
    & Methods    & Scenario 1   & Scenario 2   & Scenario 3   & Scenario 4   & Scenario 5   & Scenario 6   & Scenario 7   & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    & -            & -            & -            & -            & -            & -            & -            & -            & -            \\
  1 & softMV     & 0.1441       & 0.1908       & 0.119        & 0.1475       & 0.1613       & 0.2035       & 0.1252       & -            & -            \\
  2 & hardMV     & 0.157        & 0.1974       & 0.1235       & 0.1623       & 0.145        & 0.1989       & 0.1361       & -            & -            \\
  3 & DS         & 0.0029       & 0.0036       & 0.0046       & 0.1155       & 0.531        & 0.1394       & 0.1375       & -            & -            \\
  4 & Raykar     & 0.0065       & 0.0071       & 0.0053       & 0.0893       & 0.1387       & 0.1051       & 0.0739       & -            & -            \\
  5 & OursGlobal & 0.1078  

In [8]:

#all from softMV
aux = find_match("softMV",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="soft-MV"))

#all from hardMV
aux = find_match("hardMV",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="hard-MV"))


aux = find_match("DS",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="D&S"))

aux = find_match("Raykar",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="Raykar"))

aux = find_match("OursGlobal_",files)
for value in aux:
    plot_files(value)         
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="Ours Global"))



File to show:  LabelMe_softMV_test.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global          0.819949      0.819949
====  ============  ==========  ============

File to show:  LabelMe_softMV_train.csv
====  ============  ==========  ============  ============  =========  =============  ==========
  ..  Unnamed: 0      Accuracy    F1 (micro)    Mean NormF    Mean JS    Wmean NormF    Wmean JS
====  ============  ==========  ============  ============  =========  =============  ==========
   0  Global             0.834         0.834      0.171686   0.215826       0.116717    0.106678
====  ============  ==========  ============  ============  =========  =============  ==========
Epochs: 9.266666666666667

File to show:  LabelMe_hardMV_test.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Gl